In [1]:
%matplotlib widget
import sys
import os
import re
#sys.path = [p for p in sys.path if p.find('/opt/apps/software/') == -1]
from glob import glob
from IPython.display import display, HTML
from matplotlib import pyplot as plt

from utils.constants import UNIVARIATE_ARCHIVE_NAMES as ARCHIVE_NAMES
from utils.constants import UNIVARIATE_DATASET_NAMES as DATASET_NAMES
from utils.utils import read_all_datasets, transform_labels, create_directory, run_length_xps, generate_results_csv, plot_epochs_metric
from utils.data_loading import get_multiple_data, predict, shifted_zscore, zscore, print_metric, tf_rmse, tf_pmse, rmse
import utils
from classifiers import inception

import numpy as np
import pandas as pd
import sklearn
import keras
# keras.backend.tensorflow_backend._get_available_gpus()

# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

def tf_pmse_DA(y_true, y_pred):
    return tf_pmse(y_true, y_pred, idx=0)

def tf_pmse_5HT(y_true, y_pred):
    return tf_pmse(y_true, y_pred, idx=1)

def tf_pmse_pH(y_true, y_pred):
    return tf_pmse(y_true, y_pred, idx=2)

def tf_pmse_NE(y_true, y_pred):
    return tf_pmse(y_true, y_pred, idx=3)

# import keras
# keras.backend.tensorflow_backend._get_available_gpus()

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [2]:
## names = ['DA', '5HT', 'pH', 'NE']
# data_prefix = '/mnt/nfs/proj/in-vitro/iterate/results_014/model_style_008/training-A-RBE-97Hz/'
data_prefix = '/mnt/nfs/proj/in-vitro/iterate/results_014/model_style_008/training-B-RBE025-97Hz/'
voltammograms = glob(data_prefix + '/*MMA*/voltammograms.mat')
# probes = ['MMA003W01R04', 'MMA004W01R04', 'MMA013W01R04', 'MMA018W01R04', 'MMA019W01R04', 'MMA022W01R04', 'MMA023W01R04', 'MMA024W01R04', 'MMA025W01R04']
# probes = ['MMA003W01R04', 'MMA004W01R04', 'MMA013W01R04', 'MMA022W01R04', 'MMA025W01R04']
probes = ['MMA003W01R04', 'MMA004W01R04', 'MMA013W01R04']
good_probes = probes

hold_probe = probes[0]
# hold_probe=None
print('hold probe :', hold_probe)

output_directory = f'/mnt/nfs/proj/in-vitro/Leonardo/inception/results/mm_partial/{hold_probe}/'

if not (os.path.exists(output_directory)):
    os.makedirs(output_directory, exist_ok=True)

val_probe=None
# val_probe=probes[4]
print('validation probe :', val_probe)

print(f'Leaving out probe {hold_probe}', flush=True)
print(f'Validation probe {val_probe}', flush=True)
print(f'Loading data', flush=True)

filter_files = lambda x : x.find('500kohm') == -1
print('No 500ohms')

# filter_files = lambda x : x.find('500kohm') > -1
# print('ONLY 500ohms')

# normalize_data = minmax
# revert_data = lambda x: minmax(x, inverse=True)

normalize_data = shifted_zscore
revert_data = lambda x: shifted_zscore(x, inverse=True)

# normalize_data = zscore
# revert_data = lambda x: zscore(x, inverse=True)

# normalize_data = lambda x: x
# revert_data = lambda x: x

x_train, y_train, x_val, y_val, x_test, y_test = get_multiple_data(data_prefix,
                                                                    probes=good_probes,
                                                                    holdout=hold_probe,
                                                                    val_probe=val_probe,
                                                                    number_of_minibatches=20,
                                                                    normalize_data=normalize_data,
                                                                    filter_files=filter_files,
#                                                                     nrecords_per_session=1)
                                                                    nrecords_per_session=400)
#                                                                     ,input_interval=[200, 301])
# print('Data loaded')

if len(x_train.shape) == 2:  # if univariate
    print('adding singleton')
    # add a dimension to make it multivariate with one dimension
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))
    x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

input_shape = x_train.shape[1:]
output_shape = y_train.shape[1]


hold probe : MMA003W01R04
validation probe : None
Leaving out probe MMA003W01R04
Validation probe None
Loading data
No 500ohms
number of voltammograms files 36
after filter 36
after removing bad probes 12
number of holdout files 4
(52800, 999)
(52800, 4)
number of train/val files 8
(105600, 999)
(105600, 4)
adding singleton


In [3]:
for x in [x_train, y_train, x_val, y_val, x_test, y_test]:
    print(x.shape)
print(output_shape)
print(input_shape)

(95040, 999, 1)
(95040, 4)
(10560, 999, 1)
(10560, 4)
(52800, 999, 1)
(52800, 4)
4
(999, 1)


In [4]:
print(output_directory)

/mnt/nfs/proj/in-vitro/Leonardo/inception/results/mm_partial/MMA003W01R04/


In [5]:
# load pre model

# import keras 

# cf_model_path = '/mnt/nfs/proj/in-vitro/Leonardo/inception/results/cf/CF082/best_model.hdf5'

# # NOTE: tf_pmse* is different for CF (different mean/std). In theory it doesn't matter since we are just loading the model to get the weights, so running this model with 
# # MM data will produce wrong metrics
# cf_model = keras.models.load_model(cf_model_path, 
#                                    custom_objects={"tf_pmse_DA": tf_pmse_DA, "tf_pmse_5HT": tf_pmse_5HT, "tf_pmse_pH": tf_pmse_pH, "tf_pmse_NE": tf_pmse_NE})

cf_model = None

In [6]:
# Fit the model

classifier = inception.Regression_INCEPTION(output_directory, input_shape, output_shape, verbose=1, build=True, nb_epochs=10, 
                                            metrics=[tf_pmse_DA, tf_pmse_5HT, tf_pmse_pH, tf_pmse_NE], pre_model=cf_model,
                                            revert_data=revert_data)

model_path = classifier.output_directory + 'best_model.hdf5'
if os.path.isfile(model_path):
    print('Best model already fit: %s'%model_path)
    best_model = classifier.get_best_model()
else:
    print('Model not fit yet')
    best_model = None

# Epoch 1/1
# 369600/369600 [==============================] - 330s 894us/step - 
# loss: 1.4710 - tf_pmse_DA: 995.1803 - tf_pmse_5HT: 1049.4110 - tf_pmse_pH: 0.3064 - tf_pmse_NE: 973.5949 - 
# val_loss: 1.1462 - val_tf_pmse_DA: 680.6979 - val_tf_pmse_5HT: 671.7536 - val_tf_pmse_pH: 0.3898 - val_tf_pmse_NE: 678.4900
                                        
# 369600/369600 [==============================] - 338s 913us/step - 
# loss: 1.3524 - tf_pmse_DA: 963.2611 - tf_pmse_5HT: 989.8214 - tf_pmse_pH: 0.2679 - tf_pmse_NE: 911.7079 - 
# val_loss: 1.8146 - val_tf_pmse_DA: 973.9214 - val_tf_pmse_5HT: 1004.9336 - val_tf_pmse_pH: 0.5226 - val_tf_pmse_NE: 884.2783

INFO:tensorflow:Using MirroredStrategy with devices ('/replica:0/task:0/device:GPU:0',)
starting model from scratch...
Compiling with Adam and metrics:  ['tf_pmse_DA', 'tf_pmse_5HT', 'tf_pmse_pH', 'tf_pmse_NE']
Model not fit yet


In [7]:
# Fit inception time the model

if best_model is None:
    print('Fitting new model...')
    metrics = classifier.fit(x_train, y_train, x_val, y_val, plot_test_acc=True)
    best_model = classifier.get_best_model()
else:
    print('Model alread fit, computing prediction of validation data')
    metrics = classifier.predict(x_val, y_val, x_train, y_train, return_df_metrics=True)

display(HTML(metrics.to_html()))


Fitting new model...
mini batch size: 64
Train on 95040 samples, validate on 10560 samples
Epoch 1/10
95040/95040 [==============================] - 91s 962us/step - loss: 2.0928 - tf_pmse_DA: 1093.0405 - tf_pmse_5HT: 1099.5546 - tf_pmse_pH: 0.3546 - tf_pmse_NE: 1049.4706 - val_loss: 2.4095 - val_tf_pmse_DA: 965.4555 - val_tf_pmse_5HT: 1025.4335 - val_tf_pmse_pH: 0.6521 - val_tf_pmse_NE: 1973.8264
Epoch 2/10
95040/95040 [==============================] - 81s 857us/step - loss: 0.4549 - tf_pmse_DA: 703.6925 - tf_pmse_5HT: 733.6657 - tf_pmse_pH: 0.1907 - tf_pmse_NE: 708.7860 - val_loss: 10.5767 - val_tf_pmse_DA: 6635.3001 - val_tf_pmse_5HT: 995.7558 - val_tf_pmse_pH: 0.1682 - val_tf_pmse_NE: 620.6388
Epoch 3/10
95040/95040 [==============================] - 81s 855us/step - loss: 0.2754 - tf_pmse_DA: 559.2706 - tf_pmse_5HT: 584.1982 - tf_pmse_pH: 0.1340 - tf_pmse_NE: 566.0376 - val_loss: 1.6003 - val_tf_pmse_DA: 786.5269 - val_tf_pmse_5HT: 885.2966 - val_tf_pmse_pH: 0.1353 - val_tf_pmse_

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,951.206493,680.30204,0.155617,558.657666,827.836833


In [8]:
metrics = classifier.predict(x_test, y_test, x_train, y_train, return_df_metrics=True)
display(HTML(metrics.to_html()))

,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,1398.390018,1515.178609,0.543601,1439.381405,0.0


In [9]:
# import time
# metrics = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0],
#                    columns=['rmse_DA', 'rmse_5HT', 'rmse_pH', 'rmse_NE', 'duration'])
# metrics['rmse_DA'] = 1200
# metrics['rmse_5HT'] = 1100
# metrics['rmse_NE'] = 1000
# while (int(metrics['rmse_DA'].tolist()[0] > 700) + int(metrics['rmse_5HT'].tolist()[0] > 700) + int(metrics['rmse_NE'].tolist()[0] > 700)) > 1:
#     print(int(metrics['rmse_DA'].tolist()[0] > 700) + int(metrics['rmse_5HT'].tolist()[0] > 700) + int(metrics['rmse_NE'].tolist()[0] > 700))
#     metrics['rmse_DA'] -= 100 
#     metrics['rmse_5HT'] -= 100
#     metrics['rmse_NE'] -= 100
#     print(metrics)
#     time.sleep(.5)    

In [10]:
model_path = os.path.join(classifier.output_directory, 'best_model.hdf5')
# model_path = os.path.join(classifier.output_directory, 'last_model.hdf5')
# model_path = os.path.join(classifier.output_directory, 'model_init.hdf5')

print(model_path)

model = keras.models.load_model(model_path, custom_objects={"tf_pmse_DA": tf_pmse_DA, "tf_pmse_5HT": tf_pmse_5HT, "tf_pmse_pH": tf_pmse_pH, "tf_pmse_NE": tf_pmse_NE})

xt, yt = x_test, y_test
# xt, yt = x_val, y_val

yp = model.predict(xt, batch_size=64)

yp = np.apply_along_axis(revert_data, axis=1, arr=yp) 
yt = np.apply_along_axis(revert_data, axis=1, arr=yt) 

rmse4 = rmse(yt, yp)

metrics2 = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0], columns=['rmse_DA', 'rmse_5HT', 'rmse_pH', 'rmse_NE', 'duration'])
metrics2['rmse_DA'] = rmse4[0]
metrics2['rmse_5HT'] = rmse4[1]
metrics2['rmse_pH'] = rmse4[2]
metrics2['rmse_NE'] = rmse4[3]
metrics2['duration'] = 0.0
        
display(HTML(metrics2.to_html()))

/mnt/nfs/proj/in-vitro/Leonardo/inception/results/mm_partial/MMA003W01R04/best_model.hdf5


,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,1398.390018,1515.178609,0.543601,1439.381405,0.0


In [11]:
# def plot_concentrations(y):
#     fig, axs = plt.subplots(2, 2, sharey=False, tight_layout=True)
#     for ip in range(0,4):
#         axs[np.unravel_index(ip, axs.shape)].hist(y[:,ip])
# plot_concentrations(y_test)
# plot_concentrations(y_pred)

def plot_compare_test_pred(yt, yp):
    fig, axs = plt.subplots(2, 4, sharey=False, tight_layout=True, figsize=(12, 5))
    for ip in range(0,8):
        axsidx = np.unravel_index(ip, axs.shape)
        if ip < 4:
            axs[axsidx].hist(yt[:,ip])
        else:
            axs[axsidx].hist(yp[:,ip-4])

plot_compare_test_pred(yt, yp)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
best_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 999, 1)       0                                            
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 999, 1)       0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 999, 32)      1280        input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 999, 32)      640         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (

In [14]:
model = best_model

idxs = np.random.permutation(x_val.shape[0])
x_cam = x_val[idxs[:1000],:,:]
y_cam = y_val[idxs[:1000], :]

w_k_c = model.layers[-1].get_weights()[0] # weights for each filter k for each class c 

new_input_layer = model.inputs # same input of the original model

# new_outpu_layer = [model.get_layer("conv1d_93").output, model.layers[-1].output] # output is both the original as well as the before last layer 
new_outpu_layer = [model.get_layer("conv1d_31").output, model.layers[-1].output] # output is both the original as well as the before last layer 
# new_outpu_layer = [model.get_layer("activation_8").output, model.layers[-1].output] # output is both the original as well as the before last layer 

new_function = keras.backend.function(new_input_layer,new_outpu_layer)

new_feed_forward = new_function

[conv_out, y_pred] = new_feed_forward((x_cam,))

metrics = classifier._calculate_metrics(y_cam, y_pred, 0.0)
display(HTML(metrics.to_html()))
print(conv_out.shape)

,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,938.492114,680.213353,0.156132,543.60535,0.0


(1000, 999, 128)


In [15]:
w_k_c.shape

(128, 4)

In [16]:
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
names = ['DA', '5HT', 'pH', 'NE']

# print("original_label: "+str(encoder.inverse_transform(np.argmax(original_binary_class))))
# print("original_shape: "+str(time_series_original.shape))
# print("predicted_label:"+str(encoder.inverse_transform(np.argmax(predicted))))
# print("predicted_shape:"+str(conv_out.shape))

print(w_k_c.shape)
print(conv_out.shape)

# conv_out_0 = conv_out[0,:,:]
conv_out_0 = np.squeeze(np.mean(conv_out, axis=0))

# fig, axs = plt.subplots(2, 2, sharey=False, tight_layout=True, figsize=(12, 5))
for j in range(y_val.shape[1]):
    
    fig, axs1 = plt.subplots(figsize=(12, 5))
#     axsidx = np.unravel_index(j, axs.shape)
#     axs1 = axs[axsidx]
    
    cas = np.zeros(dtype=np.float, shape = (conv_out.shape[1]))
    for k,w in enumerate(w_k_c[:,j]):
        cas += w * conv_out_0[:,k]
    minimum = np.min(cas)
    cas = cas - minimum
    cas = cas/max(cas)
    cas = cas * 100
    cas = cas.astype(int)
    
#     axs[axsidx].plot(cas)

    y = np.squeeze(np.mean(x_cam, axis=0))
    x = np.array(range(y.shape[0]))
    dydx = cas

    # Create a set of line segments so that we can color them individually
    # This creates the points as a N x 1 x 2 array so that we can stack points
    # together easily to get the segments. The segments array for line collection
    # needs to be (numlines) x (points per line) x 2 (for x and y)
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    # Create a continuous norm to map from data points to colors
    norm = plt.Normalize(dydx.min(), dydx.max())
    lc = LineCollection(segments, cmap='jet', norm=norm)
    # Set the values used for colormapping
    lc.set_array(dydx)
    lc.set_linewidth(3)
    line = axs1.add_collection(lc)
    fig.colorbar(line, ax=axs1)

    # # Use a boundary norm instead
    # cmap = ListedColormap(['r', 'g', 'b'])
    # norm = BoundaryNorm([-1, -0.5, 0.5, 1], cmap.N)
    # lc = LineCollection(segments, cmap=cmap, norm=norm)
    # lc.set_array(dydx)
    # lc.set_linewidth(2)
    # line = axs[1].add_collection(lc)
    # fig.colorbar(line, ax=axs[1])

    axs1.set_xlim(x.min(), x.max())
    axs1.set_ylim(y.min(), y.max())
    axs1.set_title(names[j])

plt.show()



(128, 4)
(1000, 999, 128)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …